<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/load_and_compare_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.4 MB/s eta 0:00:00


In [2]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import numpy as np

import json
import re
import unidecode
import time
from tqdm import tqdm


In [4]:
!git clone 'https://github.com/mille055/Rec_Project.git'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 68 (delta 24), reused 14 (delta 4), pack-reused 0
Unpacking objects: 100% (68/68), 27.36 MiB | 5.40 MiB/s, done.


In [6]:
kaggle_cleaned = pd.read_pickle('/content/Rec_Project/data/cleaned_podcast2.pkl')

In [7]:
kaggle_cleaned

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category
0,f201172864310534ae6b6c4f8ae9b8cc,1000016089,noreply@blogger.com (Richard Ham),rahdo-talks-through,Rahdo Talks Through,"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,"[leisure, games]"
1,3cfef94cfccf27eb504bd7a6c7459636,1000035707,CityPodcast,originall,OriginALL,OriginALL este un show cu oameni inteligenti p...,3.7,3,"[careers, business]"
2,9fc35a81647e391f93211b5d11973c83,1000081327,"Jim Kettner, John Mihaly, and Scott Brown",galaktacus,Galaktacus,The name Galaktacus originates from a youthful...,5.0,20,"[film, tv]"
3,ce2c3a68dda35600671acd32a1c15fc5,1000095644,Kris Baker Dersch,no-extra-words-one-persons-search-for-story,No Extra Words one person's search for story,A flash fiction podcast featuring short storie...,5.0,11,"[performing, arts, books]"
4,aed410e50370beec62b241374abb296b,1000114875,Todd Mizener,the-talking-pictures-podcast,The Talking Pictures Podcast,Photojournalism on the radio...sort of. The ph...,5.0,9,"[arts, visual, news]"
...,...,...,...,...,...,...,...,...,...
75324,84d95344bd4f9e98afbc36f645817220,999860044,Society of Hospital Medicine,before-the-white-coat,Before The White Coat,"On Before The White Coat, SHM CEO and host Lar...",5.0,2,"[fitness, careers, business, medicine, health]"
75325,075747aa236dd5e97bfa7834a6bce607,999882879,OakGroveBaptist,oakgrovebaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1,"[spirituality, religion, christianity]"
75326,c108b5f6d6972a6e9d900d222a0e6309,999909601,"Keating Dental Arts: Dental, Dentistry, Dentist",dental-up,Dental Up,Dental Up is your source for everything dental...,5.0,21,[education]
75327,3c19556d722b83c643f2d0ff59891ec1,999962739,Sarika D. Mehta,sarika-d-mehta-podcast,Sarika D. Mehta » Podcast,Awesome interviews with fascinating people.,5.0,2,"[news, arts]"


In [9]:
siddgood = pd.read_pickle('/content/Rec_Project/data/podcast_dataframe_from_siddgood.pkl')
siddgood

,title,producer,genre,description,num_episodes,rating,num_reviews,link,episode_titles,episode_descriptions
0,History Hyenas with Chris Distefano and Yannis...,RiotCast Network,Comedy,Comedians Yannis Pappas and Chris Distefano go...,98.0,4.9,1300.0,https://podcasts.apple.com/us/podcast/history-...,robert moses was wild kinsey was wild prohibit...,the boys discuss robert moses wild the hyenas ...
1,Curiosity Daily,Westwood One,Education,The award-winning Curiosity Daily podcast from...,300.0,4.8,428.0,https://podcasts.apple.com/us/podcast/curiosit...,maps as weapons and art w justin fornal and em...,explorers justin fornal and emiliano ruprah fr...
2,Spirits,Multitude,History,"A boozy weekly podcast about mythology, legend...",161.0,4.8,1200.0,https://podcasts.apple.com/us/podcast/spirits/...,jacheongbi multimedia mythmaking with carmen m...,this story seems like we lifted it straight ou...
3,The Soundtrack Show,iHeartRadio,TV & Film,The Soundtrack Show is a weekly look at film s...,67.0,4.9,879.0,https://podcasts.apple.com/us/podcast/the-soun...,season intermission the nightmare before chri...,season intermission learn more about your ad...
4,Writing Excuses,"Brandon Sanderson, Mary Robinette Kowal, Dan W...",Arts,"Brandon Sanderson, Mary Robinette Kowal, Howar...",58.0,4.7,775.0,https://podcasts.apple.com/us/podcast/writing-...,economics realism vs ruleofcool sequencing you...,your hosts brandon dan howard and mahtab econ...
...,...,...,...,...,...,...,...,...,...,...
4298,Dark Histories,Ben Cutmore,History,Bi-weekly narratives on the unsolved and the u...,72.0,4.9,287.0,https://podcasts.apple.com/us/podcast/dark-his...,the yatton demoniac george lukins the sarah du...,sir when you can spare room in your gazette i ...
4299,Red Scare,Red Scare,Society & Culture,Red Scare is a cultural commentary podcast hos...,99.0,4.0,1200.0,https://podcasts.apple.com/us/podcast/red-scar...,bore carnival teaser pest control greta fg thu...,the ladies discuss the online ya wars the cuts...
4300,The Amelia Project,Imploding Fictions,Fiction,The Amelia Project is a secret agency that fak...,39.0,4.9,386.0,https://podcasts.apple.com/us/podcast/the-amel...,richard season finale ant cleo cole niseag b...,i always obey the dice episode season fina...
4301,L.A. Not So Confidential,Dr. Scott & Dr. Shiloh,True Crime,An L.A. based podcast brought to you by two fo...,36.0,4.8,151.0,https://podcasts.apple.com/us/podcast/l-a-not-...,killer nurses the fear of clowns intimate part...,dr scott and dr shiloh delve into the world of...


In [11]:
myscrape = pd.read_pickle('/content/Rec_Project/data/re-scraped_podcasts_with_episode_description.pkl')
myscrape

,title,producer,genre,description,num_episodes,rating,num_reviews,link,itunes_id,epidsode_description
0,Kosmographia,Randall Carlson,Science,All the Randall you can handle,95,4.9,699.0,https://podcasts.apple.com/us/podcast/kosmogra...,1479346591,[We want to make these recorded podcast more d...
1,Coffee + Crumbs Podcast,"Katie Blackburn, Jill Atogwe, Ashlee Gadd",Kids & Family,The Coffee + Crumbs Podcast is your companion ...,141,4.8,1700.0,https://podcasts.apple.com/us/podcast/coffee-c...,1093725475,"[After a loooong winter hiatus, we’re back jus..."
2,Drilled,Critical Frequency,Science,"A true-crime podcast about climate change, hos...",183,4.6,2100.0,https://podcasts.apple.com/us/podcast/drilled/...,1439735906,"[Five years ago, Kiana Wilburg was a new repor..."
3,Best Laid Plans,Sarah Hart-Unger,Education,Welcome to Best Laid Plans -- a podcast all ab...,139,4.9,535.0,https://podcasts.apple.com/us/podcast/best-lai...,1525311647,"[In this episode, Sarah comes on with answers ..."
4,Script Apart,Script Apart,TV & Film,A podcast about the first-draft secrets behind...,67,4.8,77.0,https://podcasts.apple.com/us/podcast/script-a...,1518622561,"[This week on Script Apart, the return of Mike..."
...,...,...,...,...,...,...,...,...,...,...
4042,Oxventure - A Dungeons & Dragons Podcast,Oxventure,Leisure,The Oxventurers Guild presents a DnD real play...,134,5.0,655.0,https://podcasts.apple.com/us/podcast/oxventur...,1528568221,[Edvard and Lilith investigate a sinister murd...
4043,Future of Infrastructure,Microsoft Podcasts,Technology,"Host Jeremy Goldberg, Worldwide Director of Cr...",15,NaN,NaN,https://podcasts.apple.com/us/podcast/future-o...,1674271676,[Laura Tolkoff is the Transportation Policy Di...
4044,The TWIML AI Podcast (formerly This Week in Ma...,Sam Charrington,Technology,Machine learning and artificial intelligence a...,640,4.7,370.0,https://podcasts.apple.com/us/podcast/the-twim...,1116303051,"[Today we’re joined by Anastasis Germanidis, C..."
4045,The Bestie Breakdown,The Bestie Breakdown,Kids & Family,Hi! We're Shay Shull from Mix and Match Mama a...,51,3.8,1400.0,https://podcasts.apple.com/us/podcast/the-best...,1509081920,[Did the Besties just release a second episode...


In [17]:
print(kaggle_cleaned.columns)
print(myscrape.columns)

Index(['podcast_id', 'itunes_id', 'author', 'slug', 'title', 'description',
       'average_rating', 'ratings_count', 'category'],
      dtype='object')
Index(['title', 'producer', 'genre', 'description', 'num_episodes', 'rating',
       'num_reviews', 'link', 'itunes_id', 'epidsode_description'],
      dtype='object')


# Comparison of itunes_id

In [15]:
print(kaggle_cleaned.itunes_id.nunique(), myscrape.itunes_id.nunique())

75329 4047


In [25]:
# intersection of itunes_id:
common_ids = set(kaggle_cleaned['itunes_id']).intersection(set(myscrape['itunes_id']))
print(len(common_ids))  # Count common identifier values

1287


# Get Reviews

In [20]:
### mount google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [24]:
import os
import zipfile
# Unzip the data
datapath = '/content/gdrive/MyDrive/AIPI540_RecProject/'

if not os.path.exists('./data'):
    os.makedirs('./data')

zip_ref = zipfile.ZipFile(datapath+'reviews.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [26]:
with open('/content/data/reviews.json') as f:
    reviews_df = pd.DataFrame(json.loads(line) for line in f)
reviews_df.head()

,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00


In [27]:
# intersection of podcast_id with kaggle_cleaned:
common_podcast_ids = set(kaggle_cleaned['podcast_id']).intersection(set(reviews_df['podcast_id']))
print(len(common_podcast_ids))  # Count common identifier values

27


So it seems that there are not many reviews that match podcast_id with the other kaggle dataset, and there is no itunes_id in the reviews data to otherwise match the reviews another way or with the dataset I scraped...options are to re-scrape reviews somehow or ??